<a href="https://colab.research.google.com/github/Alexbox360/Data-Science/blob/main/Copy_of_filt_mixFreqs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def filt_mixFreq(Ch, TT, FREQ):
    ### Está función debe recibir datos sin ceros en TT.
    TT1 = TT[Ch==1]

    dTT1 = np.diff(TT1)
    dTT = np.diff(TT)

    ### Obtener límites para cada bloque
    limsTT = (dTT > 0.1*1e6)
    limsTT1 = (dTT1 > 0.1*1e6)
    lims_indTT = np.where(limsTT==True)[0] + 1
    lims_indTT1 = np.where(limsTT1==True)[0] + 1

    # Número de bloques
    nsegmtsTT = len(lims_indTT) + 1
    nsegmtsTT1 = len(lims_indTT1) + 1

    ### Obtener sub-arreglos por cada bloque
    if  nsegmtsTT <= 1:
      return TT, FREQ, Ch
    subTT, subTT1, subCh = [], [], []
    subTT.append( TT[:lims_indTT[0]] )
    subTT1.append( TT1[:lims_indTT1[0]] )
    subCh.append( Ch[:lims_indTT[0]] )

    for i in range(nsegmtsTT - 2):
        llTT, slTT = lims_indTT[i], lims_indTT[i + 1]
        ll, sl = lims_indTT1[i], lims_indTT1[i + 1]
        subTT1.append( TT1[ll:sl] )
        subTT.append( TT[llTT:slTT] )
        subCh.append( Ch[llTT:slTT] )

    subTT.append( TT[lims_indTT[-1]:] )
    subTT1.append( TT1[lims_indTT1[-1]:] )
    subCh.append( Ch[lims_indTT[-1]:] )

    ### Obtiene frecuencia para cada bloque
    Freqs = np.zeros(len(subTT1))
    for i in range(len(subTT1)):
        Freqs[i] = (np.mean(np.diff(subTT1[i]))*81e-12)**-1

    ### Crea TTF: arreglo sin frecuencias incorrectas.
    mask = (Freqs < FREQ*1e6+(FREQ*1e6/100)*0.1) & (Freqs > FREQ*1e6-(FREQ*1e6/100)*0.1)

    temp = []
    for i in range(len(mask)):
        if (mask[i] == True):
            temp.append(subTT[i])
    TTF = np.concatenate(temp)

    return TTF, Freqs, subCh

In [ ]:
def regresion_lineal(x, y):
    # Calcula los coeficientes de la regresión lineal (pendiente y ordenada al origen)
    n = len(x)
    sum_x = np.sum(x)
    sum_y = np.sum(y)
    sum_xy = np.sum(x * y)
    sum_x_squared = np.sum(x**2)

    sum_x = int(sum_x)
    sum_y = int(sum_y)
    sum_xy = int(sum_xy)
    sum_x_squared = int(sum_x_squared)

    m = (n * sum_xy - sum_x * sum_y) / (n * sum_x_squared - sum_x**2)
    b = (sum_y - m * sum_x) / n

    return m, b

def coeficiente_determinacion(x, y, m, b):
    # Calcula el coeficiente de determinación (R^2)
    y_pred = m * x + b
    ss_total = np.sum((y - np.mean(y))**2)
    ss_res = np.sum((y - y_pred)**2)
    r_squared = 1 - (ss_res / ss_total)

    return r_squared

In [ ]:
for j in range(5):
  for i in range(10):
    F = np.array((j+1))
    Ch = np.genfromtxt(str(j+1)+"MCh"+str(i+1)+".txt").T  + 1
    TT = np.genfromtxt(str(j+1)+"MTT"+str(i+1)+".txt").T
    if len(Ch) == len(TT):
      TTF = TT[TT != 0]
      Ch = Ch[TT != 0]
      x = np.arange(0, len(TTF))
      m, b = regresion_lineal(x, TTF)
      r_squared = coeficiente_determinacion(x, TTF, m, b)
      if r_squared < 0.999999:
          TTF, Freqs, Ch = filt_mixFreq(Ch, TTF, F)
          x = np.arange(0, len(TTF))
          m, b = regresion_lineal(x, TTF)
          r_squared = coeficiente_determinacion(x, TTF, m, b)
          print("Para el archivo: "+str(j+1)+"MTT"+str(i+1)+" se tiene R2 = ",r_squared)
      elif r_squared >= 0.999999:
          print("Para el archivo: "+str(j+1)+"MTT"+str(i+1)+" se tiene R2 = ",r_squared)
    else:
      print("Existe una incompatibilidad critica en los archivos: "+str(j+1)+"MTT"+str(i+1)+" y "+str(j+1)+"MCh"+str(i+1)+".txt")

Para el archivo: 1MTT1 se tiene R2 =  0.9999997055329122
Para el archivo: 1MTT2 se tiene R2 =  0.999999884732538
Para el archivo: 1MTT3 se tiene R2 =  0.9999999363558633
Para el archivo: 1MTT4 se tiene R2 =  0.9999999107957964
Para el archivo: 1MTT5 se tiene R2 =  0.9999997603339104
Para el archivo: 1MTT6 se tiene R2 =  0.9999999660643467
Para el archivo: 1MTT7 se tiene R2 =  0.9999998588088203
Para el archivo: 1MTT8 se tiene R2 =  0.9999999592389341
Para el archivo: 1MTT9 se tiene R2 =  0.9999999300161103
Para el archivo: 1MTT10 se tiene R2 =  0.9999999335138119
Para el archivo: 2MTT1 se tiene R2 =  0.999999889770631
Existe una incompatibilidad critica en los archivos: 2MTT2 y 2MCh2.txt
Para el archivo: 2MTT3 se tiene R2 =  0.9999999753563646
Para el archivo: 2MTT4 se tiene R2 =  0.9999999279780606
Para el archivo: 2MTT5 se tiene R2 =  0.9999999790207386
Existe una incompatibilidad critica en los archivos: 2MTT6 y 2MCh6.txt
Para el archivo: 2MTT7 se tiene R2 =  0.999999974152336
Para 